In [4]:
import numpy as np
import cv2
import copy
from progress.bar import Bar

In [2]:
pip install video

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement video (from versions: none)
ERROR: No matching distribution found for video


In [16]:
import cv2
import numpy as np
from progress.bar import Bar
import copy

def main():
    capture = cv2.VideoCapture(r'C:\Users\nadde\Downloads\x2mate.com-Novak Djokovic .mp4')
    if not capture.isOpened():
        print("Error: Could not open video.")
        return

    # Load the Haar cascade xml file for body detection
    body_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_fullbody.xml')

    length = int(capture.get(cv2.CAP_PROP_FRAME_COUNT))

    bar = Bar('Processing Frames', max=length)

    first_iteration_indicator = 1
    accum_image = None  # Define accum_image here
    for i in range(0, length):

        ret, frame = capture.read()
        if not ret:
            print("Error: Could not read frame.")
            break

        # If first frame
        if first_iteration_indicator == 1:

            first_frame = copy.deepcopy(frame)
            height, width = frame.shape[:2]
            accum_image = np.zeros((height, width), np.uint8)  # Update accum_image here
            first_iteration_indicator = 0
        else:

            # Convert the frame to grayscale
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

            # Perform body detection
            bodies = body_cascade.detectMultiScale(gray, 1.1, 3)

            # Create an empty mask to draw the detected bodies
            body_mask = np.zeros_like(gray)

            # Draw the detected bodies on the mask
            for (x, y, w, h) in bodies:
                body_mask[y:y+h, x:x+w] = 255

            # add to the accumulated image
            accum_image = cv2.add(accum_image, body_mask)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        bar.next()

    bar.finish()

    if accum_image is None:
        print("Error: accum_image is None.")
        return

    color_image = cv2.applyColorMap(accum_image, cv2.COLORMAP_HOT)
    result_overlay = cv2.addWeighted(first_frame, 0.7, color_image, 0.7, 0)

    # save the final heatmap
    cv2.imwrite('tenn.jpg', result_overlay)

    # cleanup
    capture.release()
    cv2.destroyAllWindows()


if __name__ == '__main__':
    main()
